# part 2 word2vec

In [10]:
import pandas as pd

# 데이터 불러오기
# part 1과 달리 unlabeled 훈련 데이터를 사용

train = pd.read_csv("C:/Users/hyun/Downloads/labeledTrainData.tsv/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("C:/Users/hyun/Downloads/testData.tsv/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("C:/Users/hyun/Downloads/unlabeledTrainData.tsv/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

# 데이터 다 잘 읽었나 확인 (총 100000개)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" %(train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [11]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # 문서를 단어 시퀀스로 바꾸는 함수
    # stop words 삭제
    # 리스트 형태로 반환
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [12]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # sentence 쪼개기
    # sentece(=word 리스트) 리스트로 반환
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


### data word2vec 할 준비 완료

In [14]:
sentences = [] # sentence 리스트 초기화

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\hyun\AppData\Local\Temp\ipykernel_28880\290266847.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
C:\Users\hyun\AppData\Local\Temp\ipykernel_28880\290266847.py:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


Parsing sentences from unlabeled set


In [16]:
# part1과 다른 output 확인

print(len(sentences))

print(sentences[0])

print(sentences[1])

796172
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [30]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

####
# 위에 파라미터에서 오류!
# __init__() got an unexpected keyword argument 'size' 라는 에러가 나왔는데
# 버전 문제로, 설명은 옛날 글인데 내가 설치한 버전이 더 최신이라 생긴 문제
# size 파라미터를 vector_size로 바꿔주니 해결!
####

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2024-03-05 01:11:47,217 : INFO : collecting all words and their counts
2024-03-05 01:11:47,219 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-03-05 01:11:47,325 : INFO : PROGRESS: at sentence #10000, processed 225664 words, keeping 17775 word types


Training model...


2024-03-05 01:11:47,448 : INFO : PROGRESS: at sentence #20000, processed 451738 words, keeping 24945 word types
2024-03-05 01:11:47,560 : INFO : PROGRESS: at sentence #30000, processed 670859 words, keeping 30027 word types
2024-03-05 01:11:47,673 : INFO : PROGRESS: at sentence #40000, processed 896841 words, keeping 34335 word types
2024-03-05 01:11:47,776 : INFO : PROGRESS: at sentence #50000, processed 1116082 words, keeping 37751 word types
2024-03-05 01:11:47,878 : INFO : PROGRESS: at sentence #60000, processed 1337544 words, keeping 40711 word types
2024-03-05 01:11:47,986 : INFO : PROGRESS: at sentence #70000, processed 1560307 words, keeping 43311 word types
2024-03-05 01:11:48,090 : INFO : PROGRESS: at sentence #80000, processed 1779516 words, keeping 45707 word types
2024-03-05 01:11:48,197 : INFO : PROGRESS: at sentence #90000, processed 2003714 words, keeping 48121 word types
2024-03-05 01:11:48,304 : INFO : PROGRESS: at sentence #100000, processed 2225465 words, keeping 50

### 모델링 결과 확인

In [32]:
# 뜻이 가장 다른 단어 찾기
## 이 또한 버전 업데이트로.. 에러 발생
## 'Word2Vec' object has no attribute 'doesnt_match' 라는데
## 'doesnt_match' 함수가 Word2Vec 객체의 wv 안으로 들어감(왠만한건 다 wv 붙여야하는듯)
#

model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [33]:
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [34]:
model.wv.doesnt_match("paris berlin london austria".split())

'paris'

In [35]:
model.wv.most_similar("man")

[('woman', 0.6181036233901978),
 ('lady', 0.5941401720046997),
 ('lad', 0.5537906289100647),
 ('monk', 0.5482915043830872),
 ('guy', 0.5263317823410034),
 ('farmer', 0.5231702923774719),
 ('men', 0.5203357934951782),
 ('soldier', 0.5164754390716553),
 ('businessman', 0.5013821721076965),
 ('millionaire', 0.501152515411377)]